In [1]:
from gc2reductor import *
from c2 import *
import numpy as np

In [2]:
from colorrefinement import *
import json, csv
import networkx as nx
import numpy as np
from GC2NodeClassifier import GC2NodeClassifier

In [3]:
with open('datasets/citeseer-doc-classification/citeseer.content') as f:
    reader = csv.reader(f, delimiter='\t')
    content = list(reader)

with open('datasets/citeseer-doc-classification/citeseer.cites') as f:
    reader = csv.reader(f, delimiter='\t')
    edges = list(reader)

In [5]:
node_ids = {row[0]: i for i, row in enumerate(content)}
X = np.array([
    row[1:-1] for row in content    
], dtype=float)
y = np.array([
    row[-1] for row in content
])

adj = np.zeros((len(node_ids), len(node_ids)))
for edge in edges:
    if edge[0] in node_ids and edge[1] in node_ids:
        adj[node_ids[edge[0]], node_ids[edge[1]]] = 1
        adj[node_ids[edge[1]], node_ids[edge[0]]] = 1
np.unique(y)

array(['AI', 'Agents', 'DB', 'HCI', 'IR', 'ML'], dtype='<U6')

In [6]:
from gc2explainer import GC2Explainer
from gc2combinator import DecisionTreeCombinator
explainer = GC2Explainer(combinator=DecisionTreeCombinator(combination_size=2, depth=3),  regularization=lambda x, y: x + 0.001*np.log(y))
from cProfile import Profile
from pstats import SortKey, Stats
with Profile() as prof:
    explainer.explain(adj, X, y == 'DB', depth=2)
    (
        Stats(prof)
        .strip_dirs()
        .sort_stats(SortKey.CUMULATIVE)
        .print_stats(40)
    )

         3591250 function calls (3199284 primitive calls) in 4.527 seconds

   Ordered by: cumulative time
   List reduced from 350 to 40 due to restriction <40>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.332    1.666 base_events.py:1874(_run_once)
        2    0.000    0.000    3.332    1.666 selectors.py:319(select)
        2    0.017    0.008    3.332    1.666 selectors.py:313(_select)
        1    0.000    0.000    3.315    3.315 gc2explainer.py:23(explain)
       13    0.004    0.000    2.586    0.199 gc2combinator.py:12(combine)
        2    0.000    0.000    2.358    1.179 gc2explainer.py:128(reduce)
     10/2    0.004    0.000    2.358    1.179 gc2explainer.py:131(_reduce)
     1269    0.007    0.000    2.318    0.002 helper.py:99(boolean_combinations)
     2512    0.005    0.000    2.190    0.001 helper.py:108(negate)
11760/2512    0.064    0.000    2.059    0.001 c2.py:323(simplify)
3072/1440    0.014    0.000    

In [8]:
(y == 'DB').mean()

0.21165458937198067

In [7]:
explainer.accuracy()

0.8605072463768116

In [9]:
explainer.explanation()

(∃≥1y.E(x, y) ∧ (∃≤0y.(E(x, y) ∧ ((((¬x1868(x) ∧ x2641(x)) ∧ (x729(x) ∧ x731(x))) ∧ (∃≤0y.(E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∧ (¬x1868(x) ∧ x2641(x)))) ∧ (((¬x1868(x) ∧ x2641(x)) ∧ (x729(x) ∧ x731(x))) ∧ (∃≤0y.(E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∧ (¬x1868(x) ∧ x2641(x)))))) ∧ (∃≥1y.E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∨ (x1868(x) ∨ ¬x2641(x))))) ∨ ∃≤1y.(E(x, y) ∧ (∃≤0y.(E(x, y) ∧ ((((¬x1868(x) ∧ x2641(x)) ∧ (x729(x) ∧ x731(x))) ∧ (∃≤0y.(E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∧ (¬x1868(x) ∧ x2641(x)))) ∧ (((¬x1868(x) ∧ x2641(x)) ∧ (x729(x) ∧ x731(x))) ∧ (∃≤0y.(E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∧ (¬x1868(x) ∧ x2641(x)))))) ∧ ∃≥1y.E(x, y) ∧ (((∃≤0y.(E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∨ (¬x1868(x) ∨ x731(x))) ∧ (∃≤0y.(E(x, y) ∧ ((x729(x) ∧ x731(x)) ∧ (x2641(x) ∧ x731(x)))) ∨ (¬x1868(x) ∧ x2641(x)))) ∧ (((¬x1868(x) ∧ x2641(x)) ∧ (x729(x) ∧ x731(x))) ∨ (∃≤0y.(E(x, y) ∧ ((

In [9]:
#GuardedExistsEq(10, Var.y, E(Var.x, Var.y))
#GuardedExistsGeq(4, Var.y, Or(GuardedExistsLeq(7, Var.x, E(Var.x, Var.y)), GuardedExistsGeq(13, Var.x, E(Var.x, Var.y))))
#GuardedExistsGeq(9, Var.y, GuardedExistsLeq(13, Var.x, GuardedExistsGeq(7, Var.y, E(Var.x, Var.y))))

In [10]:
formula = GuardedExistsGeq(4, Var.y, Or(GuardedExistsLeq(7, Var.x, E(Var.x, Var.y)), GuardedExistsGeq(13, Var.x, E(Var.x, Var.y))))
graph = nx.fast_gnp_random_graph(10000, 0.001)
y = np.array([formula.evaluate(graph, i) for i in range(graph.number_of_nodes())])
y.mean()

0.5986

In [11]:
""" n = 0
for i in range(graph.number_of_nodes()):
    graph.nodes[i]["red"] = np.random.randint(0, 2)
    graph.nodes[i]["blue"] = np.random.randint(0, 2)
for i in range(n):
    for j in range(graph.number_of_nodes()):
        graph.nodes[j][f"P{i}"] = np.random.randint(0, 2)
for i in range(graph.number_of_nodes()):
    graph.nodes[i]["red_xor_blue"] = graph.nodes[i]["red"] ^ graph.nodes[i]["blue"] """

' n = 0\nfor i in range(graph.number_of_nodes()):\n    graph.nodes[i]["red"] = np.random.randint(0, 2)\n    graph.nodes[i]["blue"] = np.random.randint(0, 2)\nfor i in range(n):\n    for j in range(graph.number_of_nodes()):\n        graph.nodes[j][f"P{i}"] = np.random.randint(0, 2)\nfor i in range(graph.number_of_nodes()):\n    graph.nodes[i]["red_xor_blue"] = graph.nodes[i]["red"] ^ graph.nodes[i]["blue"] '

In [12]:
""" print(f"{"red_xor_blue"}:")
print(f"{len([node for node in graph.nodes if graph.nodes[node]["red_xor_blue"]])/graph.number_of_nodes()}") """

' print(f"{"red_xor_blue"}:")\nprint(f"{len([node for node in graph.nodes if graph.nodes[node]["red_xor_blue"]])/graph.number_of_nodes()}") '

In [13]:
from gc2explainer import GC2Explainer
explainer = GC2Explainer(regularization=lambda x, y: x + 0.0001 * np.log2(y))

In [14]:
from cProfile import Profile
from pstats import SortKey, Stats
with Profile() as prof:
    explainer.explain(nx.adjacency_matrix(graph), np.zeros((graph.number_of_nodes(), 0)), y, depth=3)
    (
        Stats(prof)
        .strip_dirs()
        .sort_stats(SortKey.CUMULATIVE)
        .print_stats(40)
    )

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [ ]:
explainer.explanation()

∃≥4x.E(y, x) ∧ (∃≥13x.E(y, x) ∨ ∃≤7x.E(y, x)))

In [ ]:
explainer._formulas[0, 1]

[⊤,
 ¬⊤,
 ∃≥10x.E(y, x),
 ∃≥8x.E(y, x),
 ∃≥7x.E(y, x),
 ∃≥9x.E(y, x),
 ∃≥13x.E(y, x),
 ∃≥11x.E(y, x),
 ∃≥14x.E(y, x),
 ∃≤9x.E(y, x),
 ∃≤7x.E(y, x),
 ∃≤6x.E(y, x),
 ∃≤8x.E(y, x),
 ∃≤12x.E(y, x),
 ∃≤10x.E(y, x),
 ∃≤13x.E(y, x),
 (∃≥10x.E(y, x) ∨ ∃≤12x.E(y, x)),
 (∃≥10x.E(y, x) ∧ ∃≤12x.E(y, x)),
 (∃≥10x.E(y, x) ∨ ∃≥11x.E(y, x)),
 (∃≥10x.E(y, x) ∧ ∃≥11x.E(y, x)),
 (∃≥10x.E(y, x) ∨ ∃≤13x.E(y, x)),
 (∃≥10x.E(y, x) ∧ ∃≤13x.E(y, x)),
 (∃≥10x.E(y, x) ∧ ∃≤7x.E(y, x)),
 (∃≥10x.E(y, x) ∨ ∃≤7x.E(y, x)),
 (∃≥10x.E(y, x) ∧ ∃≥7x.E(y, x)),
 (∃≥10x.E(y, x) ∨ ∃≥7x.E(y, x)),
 (∃≥10x.E(y, x) ∧ ∃≤8x.E(y, x)),
 (∃≥10x.E(y, x) ∨ ∃≤8x.E(y, x)),
 (∃≤12x.E(y, x) ∨ ∃≥11x.E(y, x)),
 (∃≤12x.E(y, x) ∧ ∃≥11x.E(y, x)),
 (∃≤12x.E(y, x) ∧ ∃≤13x.E(y, x)),
 (∃≤12x.E(y, x) ∨ ∃≤13x.E(y, x)),
 (∃≤12x.E(y, x) ∨ ∃≤7x.E(y, x)),
 (∃≤12x.E(y, x) ∧ ∃≤7x.E(y, x)),
 (∃≤12x.E(y, x) ∨ ∃≥7x.E(y, x)),
 (∃≤12x.E(y, x) ∧ ∃≥7x.E(y, x)),
 (∃≤12x.E(y, x) ∨ ∃≤8x.E(y, x)),
 (∃≤12x.E(y, x) ∧ ∃≤8x.E(y, x)),
 (∃≥11x.E(y, x) ∨ ∃≤13x.E(y, x)),
 (∃≥

In [ ]:
explainer.accuracy()

1.0

In [ ]:
GuardedExistsEq(10, Var.y, E(Var.y, Var.x), outgoing=False)

∃=10y.(E(y, x) ∧ E(y, x))